<a href="https://colab.research.google.com/github/ravi7299/dl_medical_applications/blob/master/Breast_cancer_classification_cancernet/Breast_cancer_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:


## load kaggle api token for downloading data from kaggle
!wget --no-check-certificate \
     https://kaggleapitoken.s3.amazonaws.com/kaggle.json 
    


#from google.colab import files
#files.upload() #upload kaggle.json


!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

--2020-05-21 10:19:36--  https://kaggleapitoken.s3.amazonaws.com/kaggle.json
Resolving kaggleapitoken.s3.amazonaws.com (kaggleapitoken.s3.amazonaws.com)... 52.216.106.75
Connecting to kaggleapitoken.s3.amazonaws.com (kaggleapitoken.s3.amazonaws.com)|52.216.106.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63 [application/json]
Saving to: ‘kaggle.json’

kaggle.json         100%[===================>]      63  --.-KB/s    in 0s      

2020-05-21 10:19:36 (1.48 MB/s) - ‘kaggle.json’ saved [63/63]



In [0]:
!ls ~/.kaggle

kaggle.json


In [0]:
##download paultimothymooney/breast-histopathology-images dataset zip in /content
!kaggle datasets download -d paultimothymooney/breast-histopathology-images -p /content --force

100% 3.09G/3.10G [00:53<00:00, 36.0MB/s]
100% 3.10G/3.10G [00:53<00:00, 62.4MB/s]


In [0]:
#unzip dataset
import os
import zipfile

local_zip = 'breast-histopathology-images.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/data/')
zip_ref.close()



In [0]:
#to view file structure
!apt-get install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (660 kB/s)
Selecting previously unselected package tree.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
#!tree --dirsfirst -L 4

In [0]:

import os

ORIG_INPUT_DATASET = "./data"

#!mkdir -p ./final_data
BASE_PATH = "./final_data"

#!mkdir -p ./final_data/training
#!mkdir -p ./final_data/validation
#!mkdir -p ./final_data/testing

TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

TRAIN_SPLIT = 0.8


VAL_SPLIT = 0.1  # remaining 0.1 data foor testing

In [0]:
from imutils import paths
import random
import shutil
import os

# list and shuffle image paths
imagePaths = list(paths.list_images(ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)

print(len(imagePaths))

# compute the training and testing split
i = int(len(imagePaths) * TRAIN_SPLIT)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]

# validation and test split
i = int(len(trainPaths) * VAL_SPLIT)
valPaths = trainPaths[:i]
trainPaths = trainPaths[i:]

# datasets building
datasets = [
	("training", trainPaths, TRAIN_PATH),
	("validation", valPaths, VAL_PATH),
	("testing", testPaths, TEST_PATH)
]

555048


In [0]:



for (dType, imagePaths, baseOutput) in datasets:

	print("[INFO] building '{}' split".format(dType))

	if not os.path.exists(baseOutput):
		print("[INFO] 'creating {}' directory".format(baseOutput))
		os.makedirs(baseOutput)

	for inputPath in imagePaths:
		filename = inputPath.split(os.path.sep)[-1]
		label = filename[-5:-4]
		
		labelPath = os.path.sep.join([baseOutput, label])
	
		if not os.path.exists(labelPath):
			print("[INFO] 'creating {}' directory".format(labelPath))
			os.makedirs(labelPath)
		
		p = os.path.sep.join([labelPath, filename])
		shutil.copy2(inputPath, p)

[INFO] building 'training' split
[INFO] 'creating ./final_data/training' directory
[INFO] 'creating ./final_data/training/0' directory
[INFO] 'creating ./final_data/training/1' directory
[INFO] building 'validation' split
[INFO] 'creating ./final_data/validation' directory
[INFO] 'creating ./final_data/validation/0' directory
[INFO] 'creating ./final_data/validation/1' directory
[INFO] building 'testing' split
[INFO] 'creating ./final_data/testing' directory
[INFO] 'creating ./final_data/testing/0' directory
[INFO] 'creating ./final_data/testing/1' directory


In [0]:
!tree  -p "./final_data" --dirsfirst --filelimit 10 

./final_data
├── [drwxr-xr-x]  testing
│   ├── [drwxr-xr-x]  0 [71691 entries exceeds filelimit, not opening dir]
│   └── [drwxr-xr-x]  1 [28256 entries exceeds filelimit, not opening dir]
├── [drwxr-xr-x]  training
│   ├── [drwxr-xr-x]  0 [183177 entries exceeds filelimit, not opening dir]
│   └── [drwxr-xr-x]  1 [72628 entries exceeds filelimit, not opening dir]
└── [drwxr-xr-x]  validation
    ├── [drwxr-xr-x]  0 [30450 entries exceeds filelimit, not opening dir]
    └── [drwxr-xr-x]  1 [12160 entries exceeds filelimit, not opening dir]

9 directories, 0 files


In [0]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import SeparableConv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

class CancerNet:
	@staticmethod
	def build(width, height, depth, classes):
		
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# CONV => RELU => POOL
		model.add(SeparableConv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# (CONV => RELU => POOL) * 2
		model.add(SeparableConv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(SeparableConv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# (CONV => RELU => POOL) * 3
		model.add(SeparableConv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(SeparableConv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(SeparableConv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(256))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		return model

Using TensorFlow backend.


In [0]:
import matplotlib
matplotlib.use("Agg")

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adagrad
from keras.utils import np_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os


NUM_EPOCHS = 10
INIT_LR = 1e-2
BS = 32


trainPaths = list(paths.list_images(TRAIN_PATH))
totalTrain = len(trainPaths)
totalVal = len(list(paths.list_images(VAL_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))

trainLabels = [int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels = np_utils.to_categorical(trainLabels)
classTotals = trainLabels.sum(axis=0)
classWeight = classTotals.max() / classTotals

trainAug = ImageDataGenerator(
	rescale=1 / 255.0,
	rotation_range=20,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.05,
	horizontal_flip=True,
	vertical_flip=True,
	fill_mode="nearest")

valAug = ImageDataGenerator(rescale=1 / 255.0)

trainGen = trainAug.flow_from_directory(
	TRAIN_PATH,
	class_mode="categorical",
	target_size=(48, 48),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)

valGen = valAug.flow_from_directory(
	VAL_PATH,
	class_mode="categorical",
	target_size=(48, 48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

testGen = valAug.flow_from_directory(
	TEST_PATH,
	class_mode="categorical",
	target_size=(48, 48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

model = CancerNet.build(width=48, height=48, depth=3,
	classes=2)
opt = Adagrad(lr=INIT_LR, decay=INIT_LR / NUM_EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# fit the model
H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // BS,
	validation_data=valGen,
	validation_steps=totalVal // BS,
	class_weight=classWeight,
	epochs=NUM_EPOCHS)

# reset the testing generator and then use our trained model to
# make predictions on the data
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // BS) + 1)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))

# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(testGen.classes, predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))



Found 255805 images belonging to 2 classes.
Found 42610 images belonging to 2 classes.
Found 99947 images belonging to 2 classes.
Epoch 1/10
7993/7993 [==============================] - 687s 86ms/step - loss: 0.3928 - accuracy: 0.8372 - val_loss: 0.2046 - val_accuracy: 0.8488
Epoch 2/10
7993/7993 [==============================] - 472s 59ms/step - loss: 0.3636 - accuracy: 0.8465 - val_loss: 0.2611 - val_accuracy: 0.8461
Epoch 3/10
7993/7993 [==============================] - 462s 58ms/step - loss: 0.3603 - accuracy: 0.8480 - val_loss: 0.9710 - val_accuracy: 0.8451
Epoch 4/10
7993/7993 [==============================] - 473s 59ms/step - loss: 0.3583 - accuracy: 0.8490 - val_loss: 0.8860 - val_accuracy: 0.8507
Epoch 5/10
7993/7993 [==============================] - 470s 59ms/step - loss: 0.3573 - accuracy: 0.8489 - val_loss: 0.1508 - val_accuracy: 0.8498
Epoch 6/10
7993/7993 [==============================] - 471s 59ms/step - loss: 0.3560 - accuracy: 0.8493 - val_loss: 0.3393 - val_accur

KeyError: ignored

In [0]:
# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()

print(H.history.keys())
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

dict_keys(['val_loss', 'val_accuracy', 'loss', 'accuracy'])


In [0]:
model.save('Breast_cancer_classification_cancernet.h5')

In [0]:
from google.colab import files

files.download("Breast_cancer_classification_cancernet.h5")


In [0]:
files.download("plot.png")